<figure>
  <IMG SRC="../img/ENCCS.jpg" WIDTH=50 ALIGN="right">
</figure>

<figure>
  <IMG SRC="../img/VeloxChem.png" WIDTH=100 ALIGN="right">
</figure>

# First VeloxChem calculation

*Roberto Di Remigio* ([@robertodr](https://github.com/robertodr))

[European National Competence Centre Sweden](https://enccs.se/)

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

We start by importing the necessary modules. We also define the `here` variable, which will point to the **current folder**.

In [25]:
from pathlib import Path

from veloxchem.mpitask import MpiTask


here = Path().parent.resolve()
inputs = here.parent / "inputs"

We want to run a restricted Kohn-Sham density functional theory (KS-DFT) test calculation on water. The input file is `water.inp`:

```text
@jobs
task: scf
@end

@method settings
basis: aug-cc-pvdz
@end

@molecule
charge: 0
multiplicity: 1
units: au
xyz:  
O   0.0   0.0   0.0
H   0.0   1.4   1.1
H   0.0  -1.4   1.1
@end 
```

and we spawn a *task* using the ``MPI_COMM_WORLD`` communicator. The ``MpiTask`` object accepts a list as first argument: 
1. The first element in the list is the input file for the task.
2. The second element in the list is the name of the output file. It can be ``None``.

In [ ]:
help(MpiTask)

In [26]:
inpfile = inputs / "water.inp"

task = MpiTask([inpfile, "water.out"])

We can now run calculations within our MPI task. We want to run a *restricted* SCF calculation, so we import the corresponding driver

In [27]:
from veloxchem.scfrestdriver import ScfRestrictedDriver


scf_drv = ScfRestrictedDriver()
scf_drv.compute(task.molecule, task.ao_basis, task.min_basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

Once the calculation is done, we can print the SCF energy from the main process in the MPI communicator:

In [28]:
from veloxchem.veloxchemlib import is_mpi_master

 
if is_mpi_master():
    print(f"SCF energy = {scf_drv.get_scf_energy():20.12f}")

SCF energy =     -76.041697549818


We can also open the `water.out` output file and analyze in detail the operations performed during the calculation. For example, the convergence reports during the SCF cycles:

```text
               Iter. | Hartree-Fock Energy | Energy Change | Gradient Norm | Max. Gradient | Density Change               
               --------------------------------------------------------------------------------------------               
                  1       -76.040225882704    0.0000000000      0.10528396      0.01411581      0.00000000                
                  2       -76.041580841174   -0.0013549585      0.02402786      0.00365910      0.03765461                
                  3       -76.041683047810   -0.0001022066      0.00880167      0.00122983      0.00802405                
                  4       -76.041694341202   -0.0000112934      0.00511177      0.00094809      0.00385443                
                  5       -76.041697479438   -0.0000031382      0.00046211      0.00008294      0.00126977                
                  6       -76.041697547984   -0.0000000685      0.00007704      0.00001088      0.00028856                
                  7       -76.041697549783   -0.0000000018      0.00000995      0.00000158      0.00004450                
                  8       -76.041697549817   -0.0000000000      0.00000212      0.00000032      0.00000530                
                  9       -76.041697549819   -0.0000000000      0.00000039      0.00000005      0.00000148                
                                                                                                                          
               *** SCF converged in 9 iterations. Time: 9.10 sec.                                                         
```